[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task1_dmrf.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [60]:
# import modules

import requests
import pandas as pd
from functools import reduce
import sqlite3

from tqdm.auto import tqdm

# 1. Сбор информации о всех строющихся объектах на сайте "наш.дом.рф"


При получении запроса и преобразовании его в json, по ключу "total" можно получить общее кол-во объектов:

In [51]:
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?sortField=objId&sortType=asc&objStatus=0'
res = requests.get(url)
total_objects = res.json().get('data').get('total')
print(f"Всего объектов: {total_objects}")

Всего объектов: 10615


In [52]:
res.json().get('data')

{'list': [{'objId': 1259,
   'developer': {'devId': 773,
    'shortName': 'КРИСТАЛЛ',
    'fullName': 'КРИСТАЛЛ',
    'groupName': 'Капитал',
    'orgForm': {'id': 4,
     'fullForm': 'Общество с ограниченной ответственностью',
     'shortForm': 'ООО'},
    'companyGroup': 5610,
    'devInn': '6215031345'},
   'rpdRegionCd': 62,
   'objAddr': 'г Рязань, улица Бугровка, д. 0',
   'objCommercNm': '"СМАРТ дом на Новоселов"',
   'objFloorMin': 27,
   'objFloorMax': 27,
   'objElemLivingCnt': 440,
   'objReady100PercDt': '2023-12-31',
   'objSquareLiving': 23985.84,
   'objCompensFundFlg': 0,
   'objProblemFlg': 0,
   'latitude': 54.6045,
   'longitude': 39.8214,
   'problemFlag': 'NONE',
   'siteStatus': 'Строящиеся',
   'objGuarantyEscrowFlg': 1,
   'residentialBuildings': 1,
   'rpdNum': '62-000075',
   'pdId': 1320,
   'buildType': 'Жилое',
   'objStatus': 0},
  {'objId': 1284,
   'developer': {'devId': 175,
    'shortName': 'РОЖДЕСТВЕНО',
    'fullName': 'РОЖДЕСТВЕНО',
    'groupName':

In [30]:
# достанем все objId
limit_ = 100
num_responses = (total_objects + limit_ - 1) // limit_  # готовая формула, вместо условия на остаток от деления
obj_Ids = []

for i in tqdm(range(num_responses)):
    offset = i * limit_
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset}&limit={limit_}&sortField=objId&sortType=asc&objStatus=0'

    res = requests.get(url)
    obj_data = res.json().get('data').get('list')

    obj_cur_ids = [x.get('objId') for x in obj_data]
    obj_Ids.append(obj_cur_ids)

obj_Ids = reduce(lambda x, y: x + y, obj_Ids)

  0%|          | 0/107 [00:00<?, ?it/s]

In [32]:
print(f"Итоговое кол-во выгруженных objId: {len(set(obj_Ids))}")

Итоговое кол-во выгруженных objId: 10615


In [46]:
# выгрузка данных по каждому objId
building_dataset = []
cnt_errors = 0

for objId in tqdm(obj_Ids):
    try:
        url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{objId}'
        obj_data = requests.get(url).json().get('data')
    except:
        # если указывать конкретную ошибку, то он её почему-то не ловит
        cnt_errors += 1
    else:
        building_dataset.append(obj_data)

  0%|          | 0/10615 [00:00<?, ?it/s]

In [50]:
print(f"Ошибок во время загрузки данных: {cnt_errors}\n"
      f"Всего удалось скачать данных: {len(building_dataset)} из {total_objects}")

Ошибок во время загрузки данных: 3
Всего удалось скачать данных: 10612 из 10615


# 2. Сохранение информации в pandas DataFrame

In [54]:
dataset = pd.json_normalize(building_dataset)

In [58]:
dataset.head()

,id,pdId,region,address,nameObj,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk,metro.colors
0,1284,1527,77,"г Москва, район внутригородское муниципальное ...","Жилой Комплекс ""Мир Митино"" (сокращенное наиме...",27.0,27.0,1115,2024-12-31,Блоки,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1366,1474,23,"г Краснодар, улица им. Героя Яцкова И.В., лите...","Микрорайон ""Губернский""",25.0,25.0,207,2025-06-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1367,1474,23,"г Краснодар, улица им. Героя Яцкова И.В., лите...","Микрорайон ""Губернский""",1.0,1.0,0,2025-06-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1368,1476,23,"г Краснодар, улица им. Героя Яцкова И.В., лите...","Микрорайон ""Губернский""",1.0,19.0,321,2025-06-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1387,1477,23,"г Краснодар, улица им. Героя Яцкова И.В., лите...","Микрорайон ""Губернский""",19.0,20.0,474,2025-06-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
print(f'Размер датасета: {dataset.shape}')

Размер датасета: (10612, 108)


# 3. Сохранение информации в excel

In [57]:
dataset.to_excel('houses_dataset.xlsx')

# 4. Сохранение информации в pickle

In [56]:
dataset.to_pickle('houses_dataset')

# 5. Сохранение информации в БД

In [70]:
conn = sqlite3.connect('houses_db')

In [77]:
# так как тип list не поддерживается sqllite3, то перевдём в тип str
dataset['photoRenderDTO'] = dataset['photoRenderDTO'].astype(str)
dataset['objectTransportInfo'] = dataset['objectTransportInfo'].astype(str)
dataset['metro.colors'] = dataset['metro.colors'].astype(str)

dataset.to_sql('houses_dataset_db', conn, if_exists='replace', index=False)
conn.commit()
conn.close()